In [ ]:
from datetime import datetime 
import numpy as np
import pandas as pd
import math
import cv2
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Dropout,Dense,Activation,Flatten
from keras.preprocessing import image
from tqdm import tqdm

In [ ]:
df = pd.read_csv('./input/labels.csv')
df.head()

In [ ]:
breeds = set(df['breed'])
n = len(breeds)
breed_to_number = dict(zip(breeds,range(n)))
number_to_breed = dict(zip(range(n),breeds))

In [ ]:
n = len(df)
input_shape = (256,256)
X_train = np.zeros((n,256,256,3),dtype = np.int8)
Y_train = np.zeros((n,len(breeds)),dtype = np.int8)
for i in tqdm(range(n)):
    X_train[i] = cv2.resize(cv2.imread('./input/train/%s.jpg'%df['id'][i]),input_shape)
    Y_train[i][breed_to_number[df['breed'][i]]] = 1

In [ ]:
Y_ = np.zeros(n,dtype = np.int8)
for i in tqdm(range(n)):
    Y_[i] = breed_to_number[df['breed'][i]]
Y_ = keras.utils.to_categorical(Y_,len(breeds))
print(Y_.shape)

In [ ]:
print(X_train.shape[1:])

In [ ]:
model = Sequential()
model.add(Conv2D(96,kernel_size=(11,11),
                activation='relu',
                strides=4,
                input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(3,3),
                      strides = 2))
model.add(Conv2D(256,kernel_size=(5,5),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),
                      strides=2))
model.add(Conv2D(384,kernel_size=(3,3),
                activation='relu'))
model.add(Conv2D(384,kernel_size=(3,3),
                activation='relu'))
model.add(Conv2D(256,kernel_size=(3,3),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),
                      strides=2))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(120,activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(),
             metrics=['accuracy'])

In [ ]:
h = model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_split=0.05)
